<a href="https://colab.research.google.com/github/aldo-arevalo/mimic-code/blob/master/notebooks/ICUglycemia/Notebooks/2_0_ara_pairing_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filter and preprocess of the dataset used for training and testing

## Overview

This notebook contains further filtering and preprocessing of the datasets used in this study.  
We we loaded the datafile __PAIR_WITH_BP.csv__ extracted in __3_get_relative_characteristics_and_make_further_filters__  .
To run this program, please change the database connection settings.



In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as colors
from scipy import stats
from datetime import datetime
import time
import warnings
#import psycopg2
# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

#conn = psycopg2.connect(database="mimiciv", user="postgres", password="tby314159", host="127.0.0.1", port="5432",options="-c search_path=mimic_icu")

def convert_to_datetime(df,time_cols):
    for t_col in time_cols:
        df[t_col] = pd.to_datetime(df[t_col],utc=True)
    
    return(df)

In [58]:
# read paired data from csv file 
origin_pair = pd.read_csv('./Data/PAIR_WITH_BP.csv')


In [59]:
origin_pair.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'icu_admissiontime',
       'icu_dischargetime', 'los_icu_days', 'timer', 'starttime', 'glctimer',
       'endtime', 'inputamount', 'input_itemid', 'patientweight', 'event',
       'glc', 'item_glc', 'glcsource', 'chart_store', 'timediff_before',
       'timediff_after', 'glc_before', 'clause_before', 'timer_before',
       'anchor_age', 'glc_after', 'timer_after', 'hasnutrison', 'ethnicity',
       'gender', 'sofa', 'creatinine', 'bun', 'bilirubin', 'bilirubin_direct',
       'bilirubin_indirect', 'alt', 'ast', 'sbp', 'dbp', 'mbp'],
      dtype='object')

In [60]:

#统计
print("Total insulin injection event: {}".format(origin_pair[origin_pair['inputamount']>0].shape[0]))
print("Of which:")
print("mathced glc_before: {}".format(origin_pair[origin_pair['glc_after']>0].shape[0]))
print("mathced glc_after: {}".format(origin_pair[origin_pair['glc_after']>0].shape[0]))
x=origin_pair[origin_pair['glc_before']>0]
y=origin_pair[origin_pair['glc_after']>0]
print("matched matched before and after: {}".format(x[x['glc_after']>0].shape[0]))
print("matched matched before and after: {}".format(y[y['glc_before']>0].shape[0]))

Total insulin injection event: 174280
Of which:
mathced glc_before: 174280
mathced glc_after: 174280
matched matched before and after: 174280
matched matched before and after: 174280


# Leave the characteristics to be the input as the network

In [61]:
data = origin_pair[["glc_before","inputamount","patientweight","anchor_age","gender","hasnutrison","ethnicity","creatinine","bun","glc_after","sbp","dbp"]]

In [65]:
data = pd.read_csv('./Data/data_with_bp.csv')

In [66]:
data['patientweight'].max(axis=0)

587.0

In [67]:
# Remove blood glucose greater than 400
data = data[(data['glc_before']<400) & (data['glc_after']<400)]
data.shape[0]

86833

In [68]:
# remove GLC_BEFORE<GLE_AFTER 
data = data [data['glc_before']>=data['glc_after']]
data.shape[0]

86833

In [69]:
data [data['hasnutrison']==1].shape[0]

0

In [70]:
# remove records with nutrison intake
data = data [data['hasnutrison']!=1]
data.shape[0]

86833

In [71]:
data['patientweight'].max(axis=0)

587.0

In [72]:
# Remove outliers with body weight less than 20
data = data[data["patientweight"]>=20]
data.shape[0]


86833

In [73]:
# Remove the item whose kidney index is null
data =data [(data['creatinine']>0) & (data['bun']>0)]
data.shape[0]


86833

In [74]:
data.gender.value_counts()

M    52891
F    33942
Name: gender, dtype: int64

In [75]:
33942/86833

0.39088825676874

In [76]:
# remove items with empty blood pressure values , 2452 items are removed
data = data[(data["sbp"]>0) & (data["dbp"]>0)]
data.shape[0]
# 这里改成86833

86833

In [77]:
data.gender.value_counts()

M    52891
F    33942
Name: gender, dtype: int64

In [78]:
data.to_csv('./Data/final_data_records.csv', index=False,encoding='utf8', header = True)

In [79]:
data["inputamount"].describe()

count    86833.000000
mean         4.692656
std          3.604231
min          0.100000
25%          2.000000
50%          4.000000
75%          6.000000
max         35.000000
Name: inputamount, dtype: float64

In [80]:
data = pd.read_csv('./Data/final_data_records.csv')

In [81]:
data.ethnicity.value_counts()

WHITE                            58136
UNKNOWN                          13982
BLACK/AFRICAN AMERICAN            8549
HISPANIC/LATINO                   3676
ASIAN                             2285
AMERICAN INDIAN/ALASKA NATIVE      205
Name: ethnicity, dtype: int64

In [82]:
data.ethnicity.value_counts()/data.shape[0]

WHITE                            0.669515
UNKNOWN                          0.161022
BLACK/AFRICAN AMERICAN           0.098453
HISPANIC/LATINO                  0.042334
ASIAN                            0.026315
AMERICAN INDIAN/ALASKA NATIVE    0.002361
Name: ethnicity, dtype: float64

In [83]:
data.shape

(86833, 12)

In [55]:

# min-max normalization and one-hot encoding of the characteristics
data_matrix = np.zeros((data.shape[0],32))
np.set_printoptions(suppress=True)


def preprocess(data):
    # Calculate the maximum, minimum and average values of the numerical characteristic in data set
    numeric_items=["glc_before","inputamount","patientweight","anchor_age","creatinine","bun","sbp","dbp","glc_after"]
    
    maximums, minimums, avgs,mid,std = data[ numeric_items].max(axis=0), data[ numeric_items].min(axis=0), \
                                 data[ numeric_items].sum(axis=0) / data[ numeric_items].shape[0],data[ numeric_items].median(),data[ numeric_items].std()
    
    # Record the normalization parameters of the characteristics, and normalize the data
    max_values = maximums
    min_values = minimums
    avg_values = avgs
    print(max_values,'\n',min_values,'\n',avg_values,'\n',mid,'\n',std)
    
    for i in range(0,8):
        #"glc_before","inputamount","patientweight","anchor_age","creatinine","bun","sbp","dbp"
        data_matrix[:data.shape[0],i] = (data[numeric_items[i]] - minimums[i]) / (maximums[i] - minimums[i])
    
    # gender
    data_matrix[:data.shape[0],8] = [(1 if row.gender=='F' else 0) for row in data.itertuples()]
    
   
    #ethnicity
    data_matrix[:data.shape[0],9:15] = pd.get_dummies(data.ethnicity).values
  
    print(pd.get_dummies(data.ethnicity).head())
    
    # square
    for i in range(0,8):
        data_matrix[:data.shape[0],i+15] = data_matrix[:data.shape[0],i] ** 2
        
     # sqrt
    for i in range(0,8):
        data_matrix[:data.shape[0],i+23] = data_matrix[:data.shape[0],i] ** 0.5
    
  
    data_matrix[:data.shape[0],31] =  data["glc_after"] 
    
    print("modify the values<0 to 0")
    for row in data_matrix:
        if row[0]<0:
            print(row)
            row[0] = 0
            row[15] = 0
            row[23] = 0
            print(row)
    print("modify over")
    print(data_matrix[0])
    print(data_matrix.shape)
    
    return  data_matrix


In [56]:
data_matrix = preprocess(data)

glc_before       399.0
inputamount       35.0
patientweight    587.0
anchor_age        91.0
creatinine        22.6
bun              283.0
sbp              228.0
dbp              156.0
glc_after        398.0
dtype: float64 
 glc_before       59.000000
inputamount       0.100000
patientweight    23.900000
anchor_age       18.000000
creatinine        0.100000
bun               2.000000
sbp              39.000000
dbp              14.714286
glc_after        17.000000
dtype: float64 
 glc_before       198.489100
inputamount        4.692656
patientweight     86.856435
anchor_age        65.351733
creatinine         1.614509
bun               34.269517
sbp              119.266791
dbp               60.204563
glc_after        155.084369
dtype: float64 
 glc_before       184.00
inputamount        4.00
patientweight     83.70
anchor_age        67.00
creatinine         1.10
bun               25.00
sbp              116.50
dbp               58.75
glc_after        146.00
dtype: float64 
 glc_before    

In [84]:
import random
# random shuffle of the dataset
index = [i for i in range(len(data_matrix))] 
set_lst=set(index)
random.shuffle(index)
print(len(index))
data_matrix = data_matrix[index]


86833


In [85]:
np.save("../../Data/alldata.npy",data_matrix)

In [86]:

#data = data.values
print(data_matrix.shape)
#Split the original data set into training set and test set
#Here, 80% of the data are used for training and 20% of the data are used for testing
#There is no intersection between the test set and the training set

test_count = int(data_matrix.shape[0] * 0.2)
train_count =data_matrix.shape[0] - test_count
print("num of train_data:{}".format(train_count))
print("num of test_data:{}".format(test_count))

train_data = data_matrix[:train_count]

test_data =data_matrix[train_count:]
print(train_data.shape,test_data.shape)
# Save training set and testing set respectively

np.save("../../Data/traindata.npy",train_data)
np.save("../../Data/testdata.npy",test_data)

(86833, 32)
num of train_data:69467
num of test_data:17366
(69467, 32) (17366, 32)


In [129]:
#train_data = np.load("./Data/traindata.npy")
#test_data = np.load("./Data/testdata.npy")